### Script to query Each items from ISM db
* Running this piece of code before starting the analysis significantly reduces the time consuming querying process.
* This script first queries the creatinine from ISM db and creates a list of encounters of interest for pedAKI.
* Then queries other features for only the encounters that have creatinine measurements
* Then saves each dataframe in a designated directory.

In [1]:
import pandas as pd
import pickle
import os
import numpy as np
import ism_utilities_Ben as ism

In [2]:
reload(ism)
ismdb = ism.queryISM()

In [3]:
fileDir = os.path.dirname('__file__')

In [4]:
# scr_dic = {'creatinine': {'itemid': [1461], 'valcol': [1], 'uomstr': ['mg/dL']}}

scrdf = ismdb.getItemData(1461, 1,'mg/dL', unique_census_time=True, in_anno=True)

C:\Users\310248864\AppData\Local\Continuum\Anaconda2\lib\site-packages\pandas\core\indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\310248864\AppData\Local\Continuum\Anaconda2\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [5]:
# scrdf.to_pickle(os.path.join(fileDir, 'item_df_ism', 'ism_creatinine_df.pkl'))
encounter_ids = np.array(scrdf.encounter_id.unique())

In [6]:
item_dic = {'hr': {'itemid': [602], 'valcol': [1], 'uomstr': ['bpm']}, 
            'nsbp': {'itemid': [764], 'valcol': [1], 'uomstr': ['mmHg']}, 
            'ndbp': {'itemid': [764], 'valcol': [2], 'uomstr': ['mmHg']},
            'spo2': {'itemid': [1065, 23540], 'valcol': [1, 1], 'uomstr': ['%', '%']},
            'pao2': {'itemid': [1591, 3103], 'valcol': [1, 1], 'uomstr': ['mmHg', 'mmHg']},
            'map': {'itemid': [746], 'valcol': [1], 'uomstr': ['mmHg']}, 
            'fio2': {'itemid': [501], 'valcol': [1], 'uomstr': ['fraction']},
            'ratio_pao2_flo2': {'itemid': [902], 'valcol': [1], 'uomstr': [None]},
            'hemoglobin': {'itemid': [1501], 'valcol': [1], 'uomstr': ['g/dL']},
            'temperature': {'itemid': [1139], 'valcol': [2], 'uomstr': ['C']},
            'wbc': {'itemid': [1725], 'valcol': [1], 'uomstr': ['K/uL']},
            'platelet': {'itemid': [1598], 'valcol': [1], 'uomstr': ['K/uL']},
            'bilirubin': {'itemid': [1379], 'valcol': [1], 'uomstr': ['mg/dL']},
            'albumin': {'itemid': [1349], 'valcol': [1], 'uomstr': ['g/dL']},
            'ph': {'itemid': [1730, 3115], 'valcol': [1, 1], 'uomstr': [None, None]},
            'urine': {'itemid': [1199], 'valcol': [1], 'uomstr': ['cc/kg/hr']},
            'potassium': {'itemid': [1604, 1605, 4174], 'valcol': [1, 1, 1], 'uomstr': ['meq/L', 'meq/L', 'meq/L']},
            'calcium': {'itemid': [1433], 'valcol': [1], 'uomstr': ['mg/dL']},
            'glucose': {'itemid': [1489, 4161, 4473], 'valcol': [1, 1, 1], 'uomstr': ['mg/dL', 'mg/dL', 'mg/dL']},
            'race': {'itemid': [1981], 'valcol': [1], 'uomstr': [None]},
            'creatinine': {'itemid': [1461], 'valcol': [1], 'uomstr': ['mg/dL']},
            'lactic_acid': {'itemid': [1531, 8657, 22851], 'valcol': [1, 1, 1], 'uomstr': ['mg/dL', 'mg/dL', 'mg/dL']},
            'bun': {'itemid': [1366], 'valcol': [1], 'uomstr': ['mg/dL']},
            }
# item_dic = {'map': {'itemid': [746], 'valcol': [1], 'uomstr': ['mmHg']}}

In [7]:
if not os.path.exists(os.path.join(fileDir, 'item_df_ism')):
        os.makedirs(os.path.join(fileDir, 'item_df_ism'))
        
for item in item_dic:
    fname = os.path.join(fileDir, 'item_df_ism', 'ism_'+item+'_df.pkl')
    if not os.path.exists(fname):
        print('Querying {}'.format(item))
        itemids = item_dic[item]['itemid']
        valcols = item_dic[item]['valcol']
        uomstrs = item_dic[item]['uomstr']

        itemdf = pd.DataFrame()
        for itemid, valcol, uomstr in zip(itemids, valcols, uomstrs):
            df = ismdb.getItemData(itemid, valcol, uom_str=uomstr, encounter_list=encounter_ids, 
                                   unique_census_time=True, in_anno=True)
            if itemdf.empty:
                itemdf = df
            else:
                itemdf = pd.concat([itemdf, df], axis=0)

            itemdf.to_pickle(fname)

Querying map
